In [ ]:
import import_ipynb
import model_trainer as mt
import tensorflow as tf
import tensorflow_io as tfio
import librosa
import musdb
import mir_eval
import numpy

In [ ]:
model = tf.keras.models.load_model('./model_800.keras')

def predict(track):
    audio = track.audio.T[:, :44100 * 60 * 7]
    
    stft, angles = mt.pre_process(
        audio, 
        return_angle=True
    )

    _, true = mt.post_process_track([
        mt.pre_process(track.targets['drums'].audio.T[:, :44100 * 60 * 7]),
        mt.pre_process(track.targets['bass'].audio.T[:, :44100 * 60 * 7]),
        mt.pre_process(track.targets['other'].audio.T[:, :44100 * 60 * 7]),
        mt.pre_process(track.targets['vocals'].audio.T[:, :44100 * 60 * 7])
    ], angles)
    
    _, pred = mt.post_process_track(model(stft, training=False), angles)
    
    return numpy.array(true), numpy.array(pred)

In [ ]:

db = musdb.DB(root="./musdb18", subsets="test")
targets = ['drums', 'bass', 'other', 'vocals']

def evaluate(track, i = 0):
    true, pred = predict(track)
    
    print(f"\nTrack {i}: {track.name}")

    for (audio, name) in zip(pred, targets):
        print(f"{name}:")
        mt.display_audio(audio)

    sdrs, sirs, sars, _ = mir_eval.separation.bss_eval_sources(true, pred)

    for (sdr, sir, sar, name) in zip(sdrs, sirs, sars, targets):
        print(f"{name}:\tSDR: {sdr:.4f}\tSIR: {sir:.4f}\tSAR: {sar:.4f}")

    with open(f"./test-evaluation-poop", "a") as eval_file:
        result = f"{track.name},"
        for (sdr, sir, sar) in zip(sdrs, sirs, sars):
            result += f"{sdr},{sir},{sar},"
        eval_file.write(result[:-1] + "\n")

for i, track in enumerate(db):
    evaluate(track, i)